In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [2]:
FEATURES = [
 "price_level", 
]

In [3]:
def extract_data(clinic, clinic_estim):
    X = [int(clinic['price_level'])]
    y=[int(clinic_estim['price'])]
    return [X, y]

In [4]:
def normale(mat):
    for i in [0,1,2,3]:
        mass = [k[i] for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [5]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [36]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    train_X = np.array(train_X).reshape(len(train_X),1)
    test_X = np.array(test_X).reshape(len(test_X),1)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    for i in regr.coef_:
        print "%.3f" % i
    print "%.3f" % regr.intercept_
    return regr

In [7]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

In [37]:
for i in range(1,5):
    print i
    regr = process_with(X_all, y_all, short=True, ts=i/10.0)
    print

1
Total: 49, train: 44, test: 5
1.582
1.664
22.486
0.017
-0.724
-0.262
6.576

2
Total: 49, train: 39, test: 10
1.607
1.524
23.084
0.010
0.006
-0.219
6.520

3
Total: 49, train: 34, test: 15
1.475
1.835
30.576
0.006
-0.001
-0.158
6.640

4
Total: 49, train: 29, test: 20
1.501
1.706
27.746
0.001
0.004
-0.062
6.467



In [38]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

Total: 49, train: 44, test: 5
Residual sum of squares: 3.17
Train absolute: 1.58
Test absolute: 1.66
Absolute to mean: 22.49%
Train variance score: 0.02
Test variance score: -0.72
-0.262
6.576


In [39]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_all)):
    print "Real: %f \t Predicted: %f" %(y_all[i], regr.predict(X_all[i]))

Real: 5.000000 	 Predicted: 6.051765
Real: 6.000000 	 Predicted: 6.576471
Real: 7.000000 	 Predicted: 6.051765
Real: 6.000000 	 Predicted: 6.051765
Real: 3.000000 	 Predicted: 6.051765
Real: 7.000000 	 Predicted: 6.314118
Real: 4.000000 	 Predicted: 5.789412
Real: 7.000000 	 Predicted: 5.789412
Real: 8.000000 	 Predicted: 6.314118
Real: 9.000000 	 Predicted: 6.314118
Real: 4.000000 	 Predicted: 6.576471
Real: 4.000000 	 Predicted: 6.051765
Real: 7.000000 	 Predicted: 6.051765
Real: 7.000000 	 Predicted: 6.051765
Real: 7.000000 	 Predicted: 6.051765
Real: 6.000000 	 Predicted: 6.051765
Real: 8.000000 	 Predicted: 6.314118
Real: 6.000000 	 Predicted: 6.314118
Real: 9.000000 	 Predicted: 6.314118
Real: 4.000000 	 Predicted: 5.789412
Real: 8.000000 	 Predicted: 6.051765
Real: 8.000000 	 Predicted: 6.576471
Real: 5.000000 	 Predicted: 6.576471
Real: 4.000000 	 Predicted: 6.051765
Real: 3.000000 	 Predicted: 6.051765
Real: 7.000000 	 Predicted: 6.051765
Real: 10.000000 	 Predicted: 6.314118


In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0]
print regr.intercept_

In [ ]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
X_all

In [ ]:
ind = X_all.index([0,0,0,0])
X_all.pop(ind)
y_all.pop(ind)

In [ ]:
coef = np.array([-0.7824,0.2034,0.0096,0.077,8.4162])
for x in X:
    print sum(x*coef[:-1])+coef[-1]


In [ ]:
plt.plot([i[22] for i in X])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)